In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('sample app 1').getOrCreate()

In [2]:
sc = spark.sparkContext

In [3]:
print(sc)

<SparkContext master=local appName=sample app 1>


In [26]:
data = [
    ('kuldeep','kuldeepdhadhwal@gmail.com',9878787),
    ('kuldeep','kuldeepdhadhwal@gmail.com',9878786),
    ('kuldeep','kuldeepthakur1@gmail.com',9878787),
    ('kuldeep','kuldeepthakur1@gmail.com',9878787),
    ('kuldeep','aja@gmail.com',987877),
    ('kuldeep','aja@gmail.com',987877),
    
]

In [40]:
df = spark.createDataFrame(data=data, schema=['name','email','mno'])

In [41]:
df.show()

+-------+--------------------+-------+
|   name|               email|    mno|
+-------+--------------------+-------+
|kuldeep|kuldeepdhadhwal@g...|9878787|
|kuldeep|kuldeepdhadhwal@g...|9878786|
|kuldeep|kuldeepthakur1@gm...|9878787|
|kuldeep|kuldeepthakur1@gm...|9878787|
|kuldeep|       aja@gmail.com| 987877|
|kuldeep|       aja@gmail.com| 987877|
+-------+--------------------+-------+



In [44]:
from pyspark.sql.functions import concat,col

In [45]:
df2 = df.withColumn("concat_col", concat(col("email"),col("mno")))

In [35]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,dense_rank

In [46]:
w = Window.partitionBy('concat_col').orderBy('name')

In [29]:
df = df.groupBy('email','mno').count().show()

+--------------------+-------+-----+
|               email|    mno|count|
+--------------------+-------+-----+
|kuldeepdhadhwal@g...|9878786|    1|
|       aja@gmail.com| 987877|    2|
|kuldeepdhadhwal@g...|9878787|    1|
|kuldeepthakur1@gm...|9878787|    2|
+--------------------+-------+-----+



In [55]:
df3 = df2.withColumn("row_number", row_number().over(w)).where(col("row_number")==1).drop(col("concat_col")).show()

+-------+--------------------+-------+----------+
|   name|               email|    mno|row_number|
+-------+--------------------+-------+----------+
|kuldeep|kuldeepdhadhwal@g...|9878787|         1|
|kuldeep|kuldeepdhadhwal@g...|9878786|         1|
|kuldeep|       aja@gmail.com| 987877|         1|
|kuldeep|kuldeepthakur1@gm...|9878787|         1|
+-------+--------------------+-------+----------+



In [10]:
df = df.dropDuplicates()

In [11]:
df.show()

+-------+--------------------+-------+
|   name|               email|    mno|
+-------+--------------------+-------+
|kuldeep|       aja@gmail.com| 987877|
|kuldeep|kuldeepdhadhwal@g...|9878787|
|kuldeep|kuldeepdhadhwal@g...|9878786|
|kuldeep|kuldeepthakur1@gm...|9878787|
+-------+--------------------+-------+



In [13]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8],4).map(lambda x:x+10)
print(rdd.collect())
print(rdd.getNumPartitions())

[11, 12, 13, 14, 15, 16, 17, 18]
4


In [7]:
print(rdd.first())

1


In [9]:
rdd = spark.sparkContext.parallelize((0,20))
print("From local[5]"+str(rdd.getNumPartitions()))

From local[5]1


In [10]:
rdd1 = spark.sparkContext.parallelize((0,25), 6)
print("parallelize : "+str(rdd1.getNumPartitions()))

parallelize : 6


In [11]:
rdd1.saveAsTextFile("/home/kuldeep/spark/Projects/partition")

In [ ]:
rdd2 = rdd1.repartition(4)
print("Repartition size : "+str(rdd2.getNumPartitions()))
rdd2.saveAsTextFile("/tmp/re-partition")

In [32]:
data = [('James','Smith','M',30),
  ('Anna','Rose','F',41),
  ('Robert','Williams','M',62), 
]

columns = ["firstname","lastname","gender","salary"]
df = spark.createDataFrame(data= data,schema = columns)

In [33]:
df.show()

+---------+--------+------+------+
|firstname|lastname|gender|salary|
+---------+--------+------+------+
|    James|   Smith|     M|    30|
|     Anna|    Rose|     F|    41|
|   Robert|Williams|     M|    62|
+---------+--------+------+------+



In [30]:
rdd2=df.rdd.map(lambda x: 
    (x[0]+","+x[1],x[2],x[3]*2)
    )  
df2=rdd2.toDF(["name","gender","new_salary"]   )
df2.show()

+---------------+------+----------+
|           name|gender|new_salary|
+---------------+------+----------+
|    James,Smith|     M|        60|
|      Anna,Rose|     F|        82|
|Robert,Williams|     M|       124|
+---------------+------+----------+



In [42]:
a = "test string"
result = list(map((lambda a: a.split(' ')),a))
print(result)

[['t'], ['e'], ['s'], ['t'], ['', ''], ['s'], ['t'], ['r'], ['i'], ['n'], ['g']]


In [44]:
a = [1,3,5,7]
result = list(filter(lambda x: x>3,a))
print(result)

[5, 7]


In [45]:
from functools import reduce
result = reduce(lambda x,y: x+y,a)
print(result)

16
